**热门超跌上涨**

### 选股
交易日前一天进行选股。

特征为：当日下跌（或者涨幅小于3%），高点应该**5日之前**出现，高点到当日，回撤**2%~10%**。最高点前连续上涨（红盘或大于-0.5%），**累计上涨应大于10%**，注意一定**要有一定的涨幅**。

如2023/03/29 的688313。

选择出来的股票隔日不一定上涨，但一定有一定数量是上涨股票，**实战时应看次日的表现**。

### 实战
策略应该在**10:20**左右开始运行，交易行为发生约在：10:30。

- 20cm票的涨幅不应高于5%，回撤幅度应不大于2%，10cm票涨幅不超过3%，回撤不大于1%，且当前不是最低。最好不要出现绿盘。
- **缓慢上涨**的票大涨概率大，**最高点2%的**，也有一定作为的概率，最高点少于1.5%的，有作为的概率可能低。
- **k线上最近有大涨的**，有作为的概率大。
- 10:20之前**强拉升的**，高度注意回落，缓慢拉升的不在此列。
- 20cm股票，张7到5个点，容易20cm

In [1]:
from datetime import datetime
from winq import *
import hiq_pyfetch as fetch
from winq.selector import *
db = default(log_level='info')

run_task_count = 50

base_path = '/Users/luoguochun/Downloads/stock'

In [4]:
# 清空结果集
import os
import shutil
for p in ['analyze', 'trade']:
    path = os.sep.join([base_path, p])
    files = os.listdir(path)
    for file in files:
        if file.startswith('.'):
            continue
        p = os.sep.join([path, file])
        if os.path.isdir(file):
            print('rm dir: {}'.format(p))
            shutil.rmtree(p)
        else:
            print('rm file: {}'.format(p))
            os.remove(p)
            

rm file: /Users/luoguochun/Downloads/stock/analyze/supper_falling_20230403.txt


In [2]:
# 记得改时间
test_end_date = '20230410'
with_rt_kdata = False

strategy = strategies['test.SuperFalling'](db=db,
                                           min_trade_days=30,
                                        test_end_date=test_end_date, with_rt_kdata = with_rt_kdata,
                                        run_task_count=run_task_count,
                                        load_daily=db.load_stock_daily, 
                                        load_info=db.load_stock_info,
                                        fetch_daily=fetch.fetch_stock_bar
                                       )
await strategy.prepare(max_first_day_rise=3.0,
                       max_first_day_down=-6.0,
                       max_down_as_rise=-1.5,
                       max_meet_high_days=8,
#                        max_meet_high_days=5,
                       min_high_falling=2.0,
                       max_high_falling=10.0,
                       max_turnover=15.0,
                       min_rise_to_high_ndays=10.0,
                       max_rise_unfill_gaps=5.0,
                       sort_by='chg_pct')

data = await strategy.run()

if data is not None and  len(data) > 0:
    test_end_date = strategy.test_end_date.strftime('%Y%m%d')
    path='{}/analyze/supper_falling_{}.txt'.format(base_path, test_end_date)
    export_codes_to_path(path=path, data=data)
    print('saved file: {}'.format(path))

# data
print('hit data size: {}'.format(len(data)))

处理 sh603183(建研院): 100%|██████████| 4952/4952 [01:55<00:00, 42.99it/s]    


saved file: /Users/luoguochun/Downloads/stock/analyze/supper_falling_20230410.txt
hit data size: 134


In [6]:
if data is not None and  len(data) > 0:
    test_end_date = strategy.test_end_date.strftime('%Y%m%d')
    data20cm = data[data['code'].str.startswith('sh68') | data['code'].str.startswith('sz30')]
    path='{}/analyze/supper_falling_20cm_{}.txt'.format(base_path, test_end_date)
    export_codes_to_path(path=path, data=data20cm)
    print('saved file: {}\nhit data size: {}\n'.format(path, len(data20cm)))

saved file: /Users/luoguochun/Downloads/stock/analyze/supper_falling_20cm_20230410.txt
hit data size: 54



# 实战之前应该仔细看，切记仔细看
策略应该在**10:20**左右开始运行，交易行为发生约在：10:30。

- 20cm票的涨幅不应高于10%，**5%~7%最佳**，回撤幅度应不大于2%，10cm票涨幅不超过3%，回撤不大于1%，且当前不是最低。最好不要出现绿盘。
- **缓慢上涨**的票大涨概率大，**最高点2%的**，也有一定作为的概率，最高点少于1.5%的，有作为的概率可能低。
- **k线上最近有大涨的**，有作为的概率大。
- 10:20之前**强拉升的**，高度注意回落，缓慢拉升的不在此列。
- **拉高回落**的票，大概率是出货，**不要买不要买不要买**
- 开盘前5分钟，**波动比较大，2%之内都正常**，**不要太着急卖**，太大波动才考虑卖，不要犹豫，狠心卖


In [4]:
test_end_date = None
with_rt_kdata = True

pool_date = '20230410'
is_20cm = True

# 记得修改时间
path='{}/analyze/supper_falling_{}.txt'.format(base_path, pool_date)
if is_20cm:
    path='{}/analyze/supper_falling_20cm_{}.txt'.format(base_path, pool_date)


codes=None
with open(path) as f:
    codes = f.read()
    codes = codes.split(',')
    codes = ['sh'+code if code.startswith('6') else 'sz'+code for code in codes]
    
print('loaded file: {}\ndata size: {}'.format(path, len(codes)))


strategy = strategies['test.SimpleRise'](db=db, 
                                        test_end_date=test_end_date, with_rt_kdata = with_rt_kdata,
                                        run_task_count=run_task_count,
                                        load_daily=db.load_stock_daily, 
                                        load_info=db.load_stock_info,
                                        fetch_daily=fetch.fetch_stock_bar
                                       )
await strategy.prepare(cal_time='10:30:00',
                       max_turnover=7.0,
                       max_down_as_rise=-1.0,
#                        max_20cm_rise=5.0,
                       max_20cm_rise=7.0,
                       max_20cm_falling=2.0,
                       max_10cm_rise=3.0,
                       max_10cm_falling=1.50,
                       sort_by='chg_pct')

data = await strategy.run(codes=codes)

if data is not None and  len(data) > 0:
    test_end_date = strategy.test_end_date.strftime('%Y%m%d')
    path='{}/trade/simple_rise_{}.txt'.format(base_path, test_end_date)
    if is_20cm:
        path='{}/trade/simple_rise_20cm_{}.txt'.format(base_path, test_end_date)
    export_codes_to_path(path=path, data=data)
data

处理 sh688004(博汇科技):   0%|          | 0/61 [00:00<?, ?it/s]

loaded file: /Users/luoguochun/Downloads/stock/analyze/supper_falling_20cm_20230407.txt
data size: 61


处理 sz301263(泰恩康): 100%|██████████| 61/61 [00:34<00:00,  1.78it/s]   


,code,name,falling_chg_pct,chg_pct,low_chg_pct,high_chg_pct
0,sh688256,寒武纪,1.66,4.43,-0.48,6.09
1,sz301263,泰恩康,0.66,3.55,1.36,4.21
2,sz300430,诚益通,1.02,3.13,0.34,4.15
3,sz301155,海力风电,1.81,2.84,1.36,4.65
4,sz300228,富瑞特装,1.06,2.81,0.53,3.87
5,sz300153,科泰电源,1.04,2.60,1.69,3.64
6,sh688114,华大智造,0.81,2.34,-0.29,3.15
7,sh688273,麦澜德,0.72,1.93,0.00,2.65
8,sz300461,田中精机,0.65,1.88,0.59,2.53
9,sz300059,东方财富,0.55,1.77,-0.54,2.32
